### **Reading Raw ingested data from bronze layer**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
df_ord_details = spark.read.format("parquet")\
                      .load("abfss://bronze@stretailenvdev.dfs.core.windows.net/f_OrderDetails")

In [0]:
df_ord_details.display()

### **Incremental loading**

In [0]:
silver_path = "abfss://silver@stretailenvdev.dfs.core.windows.net/s_OrderDetails"

from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark,silver_path):
    max_order_id = spark.read.format("delta")\
                        .load(silver_path)\
                        .agg(max(col("OrderDetailID")).alias("max_order_id"))\
                        .first()["max_order_id"]
else:
    max_order_id = None

In [0]:
if max_order_id :
    silver_ready = df_ord_details.filter(col("OrderDetailID") > max_order_id)
else:
    silver_ready = df_ord_details

In [0]:
print(max_order_id)

### **Upsert (Merge) logic into Silver Delta Table**

In [0]:
if DeltaTable.isDeltaTable(spark, silver_path):
    silver_tbl = DeltaTable.forPath(spark, silver_path)
    (
        silver_tbl.alias("tgt")
        .merge(
            silver_ready.alias("src"),
            "tgt.OrderDetailID = src.OrderDetailID"  # Business key
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
else:
    silver_ready.write.format("delta").mode("overwrite").save(silver_path)